In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn import cluster
import os
import re
# import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
os.getcwd()


'c:\\Users\\dscshap3808\\Documents\\my_scripts_new\\play1'

In [37]:
df = pd.read_csv('./daily_ou_kpi.csv')
# df.dropna(axis = 1, inplace = True, how = 'all')
# re1 = re.compile(r'(?<=\.).+')
# df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]
# df.to_csv('./dws_dsc_wh_ou_daily_kpi_july.csv', encoding='utf_8_sig')

# df.to_csv('./daily_kpi_all_810.csv', encoding='utf_8_sig')

In [38]:
re1 = re.compile(r'(?<=\.).+')
df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]

In [68]:
df[df['ou_code'] == 'CN-305'].head(5)

,ou_code,operation_day,inbound_receive_qty,is_holiday,outbound_shipped_qty,total_head_count,total_working_hour,outsource_working_hour,perm_working_hour,other_working_hour,direct_working_hour,indirect_working_hour,outbound_inbound_qty_ratio,perm_working_hour_ratio,working_hour_per_head,location_usage_rate,location_idle_rate
17195,CN-305,20210605,7780.0,1,39.0,20.0,240.0,0.0,0.0,0.0,240.0,0.0,0.005013,0.000000,12.000000,0.424214,0.575786
17196,CN-305,20210502,362.0,1,5.0,8.0,102.5,0.0,0.0,0.0,102.5,0.0,0.013812,0.000000,12.812500,0.000000,0.000000
17197,CN-305,20210805,23597.0,0,30567.0,21.0,246.0,200.5,45.5,0.0,246.0,0.0,1.295377,0.184959,11.714286,0.842192,0.157808
17198,CN-305,20210823,9412.0,0,13008.0,21.0,258.5,204.0,54.5,0.0,258.5,0.0,1.382065,0.210832,12.309524,0.835157,0.164843
17199,CN-305,20210626,0.0,1,42.0,15.0,182.0,171.5,10.5,0.0,182.0,0.0,0.000000,0.057692,12.133333,0.622566,0.377434


In [40]:

# clean_df2 = df.groupby('ou_code')[[
#     'inbound_receive_qty', 'outbound_shipped_qty'
#     ]].sum().reset_index()
# clean_df2['sum'] = clean_df2.sum(axis = 1)
# clean_df2 = clean_df2[clean_df2['sum'] != 0]
# df = df[df['ou_code'].isin(clean_df2.ou_code)]

In [41]:

clean_df1 = (df.groupby('ou_code')['operation_day'].count() <2).reset_index()
clean_df1.columns = ['ou_code', 'flag1']
df = clean_df1.merge(df, on = 'ou_code', how = 'inner')
df = df[df['flag1'] == False]

clean_df2 = df.groupby('ou_code')[[
    'inbound_receive_qty', 'outbound_shipped_qty'
    ]].sum().reset_index()
clean_df2['sum'] = clean_df2.sum(axis = 1)
clean_df2 = clean_df2[clean_df2['sum'] != 0]
df = df[df['ou_code'].isin(clean_df2.ou_code)]

clean_df3 = (df.groupby('ou_code')[[
    'total_working_hour'
    ]].sum() == 0).reset_index()
clean_df3 = clean_df3[clean_df3['total_working_hour'] == False]


# clean_df4 = (df.groupby('ou_code')[[
#     'outsource_working_hour'
#     ]].sum() == 0).reset_index()
# clean_df4 = clean_df4[clean_df4['outsource_working_hour'] == False]
# df = df[df['ou_code'].isin(clean_df4.ou_code)]
# df= df.reset_index()
 
# df = df[df['ou_code'].isin(clean_df3.ou_code)]
# df= df.reset_index()

df = df[[
    'ou_code','operation_day', 'inbound_receive_qty', 'is_holiday',
    'outbound_shipped_qty','total_head_count','total_working_hour',
    'outsource_working_hour', 'perm_working_hour',
    'other_working_hour', 'direct_working_hour', 'indirect_working_hour',
    'outbound_inbound_qty_ratio', 'perm_working_hour_ratio',
    'working_hour_per_head', 'location_usage_rate', 'location_idle_rate']]
df = df.fillna(0)
df = df[df['total_working_hour'] != 0]
df.head()

,ou_code,operation_day,inbound_receive_qty,is_holiday,outbound_shipped_qty,total_head_count,total_working_hour,outsource_working_hour,perm_working_hour,other_working_hour,direct_working_hour,indirect_working_hour,outbound_inbound_qty_ratio,perm_working_hour_ratio,working_hour_per_head,location_usage_rate,location_idle_rate
0,CN-001,20210430,0.0,0,1699.0,4.0,32.75,0.0,0.00,0.0,32.75,0.0,0.000000,0.0,8.1875,0.000000,0.000000
1,CN-001,20210619,0.0,1,0.0,3.0,30.00,0.0,30.00,0.0,30.00,0.0,0.000000,1.0,10.0000,0.011628,0.988372
2,CN-001,20210427,4794.0,0,7392.0,4.0,32.28,0.0,0.00,0.0,32.28,0.0,1.541927,0.0,8.0700,0.000000,0.000000
3,CN-001,20210419,11400.0,0,6000.0,4.0,33.25,0.0,0.00,0.0,33.25,0.0,0.526316,0.0,8.3125,0.000000,0.000000
4,CN-001,20210617,114.0,0,13527.0,5.0,40.58,0.0,40.58,0.0,40.58,0.0,118.657895,1.0,8.1160,0.001600,0.998400


In [42]:
len(df['ou_code'].unique())

65

In [23]:
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans 

In [72]:
# from sklearn.metrics import davies_bouldin_scores

def mnb_kmeans_in(ou_code):
        """
        mini batch kmeans, inbound, outbound, working hour data.
        simple algorithm, adding cols {max, min, mean, median, 75 quantile, distance to kernal}
        Calculate Davies Bouldin score
        from sklearn.metrics import davies_bouldin_score
        null data fill
        """

        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'inbound_receive_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
        df_fin['kernal_core1' ] = -1
        df_fin['kernal_value1'] = 0

        """
        not null data training
        """
        df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      
        scores = []
        for center in list(range(2,15)):
            kmeans = cluster.KMeans(n_clusters=center)
            model  = kmeans.fit_predict(
                        X = np.reshape(list(df_rec['inbound_receive_qty']), (-1,1))
                        )
            score  = davies_bouldin_score(
                        np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)), model
                        )
            scores.append(score)

        center = np.argmin(scores)
        print(center)
        alg1 = cluster.MiniBatchKMeans(n_clusters = center, random_state = 529)
        hist1 = alg1.fit(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
        df_rec['kernal_core1'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,center))], axis = 1
                )
        
        cl_1.columns = ['kernal_value1', 'kernal_core1']
        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core1', how = 'inner'
                )
        
        """
        merging
        """
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)

        # df_fin['kind'] = 'inbound'

        return df_fin



# scores = []
# centers = list(range(2,30))
# for center in centers:
#     scores.append(get_kmeans_score(cluster_df, center))
 

In [73]:
mnb_kmeans_in('CN-214')

12


,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1
0,CN-214,20210815,0.0,-1,0.00000
1,CN-214,20210717,0.0,-1,0.00000
2,CN-214,20210719,0.0,-1,0.00000
3,CN-214,20210729,0.0,-1,0.00000
4,CN-214,20210711,0.0,-1,0.00000
...,...,...,...,...,...
146,CN-214,20210426,15269.0,8,14871.62605
147,CN-214,20210707,72785.0,2,75275.40000
148,CN-214,20210901,76181.0,2,75275.40000
149,CN-214,20210609,88250.0,11,88250.00000


In [25]:
# df_fin = pd.DataFrame()
# df_sub = df[df['ou_code'] == 'CN-305'][['ou_code', 'operation_day', 'inbound_receive_qty']]        
# df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
# df_fin['kernal_core1' ] = -1
# df_fin['kernal_value1'] = 0
# """
# not null data training
# """
# df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      
# scores = []
# for center in list(range(2,10)):
#     kmeans = cluster.KMeans(n_clusters=center)
#     model  = kmeans.fit_predict(X = np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
#     score  = davies_bouldin_score(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)), model)
#     scores.append(score)
# center = min(scores)
# np.argmin(scores)

0

In [46]:
mnb_kmeans_in('CN-305')

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1
0,CN-305,20210626,0.0,-1,0.00000
1,CN-305,20210429,0.0,-1,0.00000
2,CN-305,20210730,0.0,-1,0.00000
3,CN-305,20210827,0.0,-1,0.00000
4,CN-305,20210528,0.0,-1,0.00000
...,...,...,...,...,...
146,CN-305,20210715,45430.0,4,48504.32243
147,CN-305,20210806,35732.0,4,48504.32243
148,CN-305,20210712,44814.0,4,48504.32243
149,CN-305,20210813,35743.0,4,48504.32243


In [45]:

def mnb_kmeans_in(ou_code):
        """
        mini batch kmeans, inbound, outbound, working hour data.
        simple algorithm, adding cols {max, min, mean, median, 75 quantile, distance to kernal}
        """
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        """
        null data fill
        """
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'inbound_receive_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['inbound_receive_qty'] == 0])
        df_fin['kernal_core1' ] = -1
        df_fin['kernal_value1'] = 0
        """
        not null data training
        """
        df_rec = df_sub[df_sub['inbound_receive_qty'] != 0]      
        hist1 = alg1.fit(np.reshape(list(df_rec['inbound_receive_qty']), (-1,1)))
        df_rec['kernal_core1'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value1', 'kernal_core1']
        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core1', how = 'inner'
                )
        
        """
        merging
        """
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)

        # df_fin['kind'] = 'inbound'

        return df_fin


def mnb_kmeans_out(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'outbound_shipped_qty']]        
        df_fin = df_fin.append(df_sub[df_sub['outbound_shipped_qty'] == 0])
        df_fin['kernal_core2' ] = -1
        df_fin['kernal_value2'] = 0
        df_rec = df_sub[df_sub['outbound_shipped_qty'] != 0]

        hist1 = alg1.fit(np.reshape(list(df_rec['outbound_shipped_qty']), (-1,1)))

        df_rec['kernal_core2'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value2', 'kernal_core2']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core2', how = 'inner'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        # df_fin['kind'] = 'outbound'
        return df_fin

def mnb_kmeans_hr(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'total_working_hour']]        
        df_fin = df_fin.append(df_sub[df_sub['total_working_hour'] == 0])
        df_fin['kernal_core3' ] = -1
        df_fin['kernal_value3'] = 0
        df_rec = df_sub[df_sub['total_working_hour'] != 0]

        hist1 = alg1.fit(np.reshape(list(df_rec['total_working_hour']), (-1,1)))

        df_rec['kernal_core3'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value3', 'kernal_core3']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core3', how = 'inner'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        """
        组内kernal distance 
        """
        df_fin['dis_core']=df_fin.groupby(
                'kernal_core3'
                )['total_working_hour','kernal_value3'].diff(
                        axis = 1
                        ).drop('total_working_hour', axis = 1).round(3)
        
        return df_fin



In [45]:
len(df['ou_code'].unique())


65

In [47]:
ou_codes = list(df['ou_code'].unique())
p = list()
for i in ou_codes:
    try: 
        mnb_kmeans_in(i)
        mnb_kmeans_out(i)
        mnb_kmeans_hr(i)
        # mnb_kmeans_hr2(i)
    except:
        
        p.append(i)


for i in p:
    ou_codes.remove(i)
[p, len(p), len(ou_codes)]

[['CN-070', 'CN-392', 'HK-015'], 3, 62]

In [48]:
%%time
from functools import reduce
df_final = pd.DataFrame()
for i in ou_codes:
        df_final = df_final.append(
            reduce(
                lambda left,right: pd.merge(
                    left,right,on= ['ou_code', 'operation_day']
                ), [mnb_kmeans_in(i), mnb_kmeans_out(i), mnb_kmeans_hr(i)]
                )
    )

"""
for loop , 对所有ou进行独立的kmeans on inb qty and outb qty
随后merge 原始表
"""

# np.setdiff1d(ou_codes,df_final.ou_code.unique())
df_final = df_final.merge(
    df[['ou_code','operation_day',\
        'outbound_inbound_qty_ratio','working_hour_per_head','total_head_count','is_holiday']],
    on = ['ou_code', 'operation_day'],
    how = 'left'
    )


Wall time: 20.6 s


In [48]:
# df_final.groupby(
#                 'kernal_core3'
#                 )['total_working_hour','kernal_value3'].diff(axis = 1).drop('total_working_hour', axis = 1)

In [49]:
# df_final['outsource_working_hour']

In [55]:
def mnb_kmeans_hr2(ou_code):
        alg1 = cluster.MiniBatchKMeans(n_clusters = 5, random_state = 5290403)
        df_fin = pd.DataFrame()
        df_sub = df[df['ou_code'] == ou_code][['ou_code', 'operation_day', 'outsource_working_hour']]  
        df_fin = df_fin.append(df_sub[df_sub['outsource_working_hour'] == 0])
        df_fin['kernal_core4' ] = -1
        df_fin['kernal_value4'] = 0
        df_rec = df_sub[df_sub['outsource_working_hour'] != 0]
 
        hist1 = alg1.fit(np.reshape(list(df_rec['outsource_working_hour']), (-1,1)))

        df_rec['kernal_core4'] = hist1.labels_
        cl_1 = pd.concat(
                [pd.DataFrame(hist1.cluster_centers_), pd.Series(np.arange(0,5))], axis = 1
                )
        
        cl_1.columns = ['kernal_value4', 'kernal_core4']

        df_rec = df_rec.merge(
                cl_1, on = 'kernal_core4', how = 'outer'
                )
        df_fin = df_fin.append(df_rec).reset_index().drop(['index'], axis = 1)
        
        df_fin['dis_core_os']=df_fin.groupby(
                'kernal_core4'
                )['outsource_working_hour','kernal_value4'].diff(
                        axis = 1
                        ).drop('outsource_working_hour', axis = 1).round(3)
        return df_fin


ou_codes
p = list()
for i in ou_codes:
    try: 
        # mnb_kmeans_in(i)
        # mnb_kmeans_out(i)
        # mnb_kmeans_hr(i)
        mnb_kmeans_hr2(i)
    except:
        
        p.append(i)


for i in p:
    ou_codes.remove(i)
 

In [56]:
[len(p),len(ou_codes)]

[0, 33]

In [57]:
df_final2 = pd.DataFrame()
for i in ou_codes:
        df_final2 = df_final2.append(
            mnb_kmeans_hr2(i)
        )

df_final = df_final.merge(df_final2, on =  ['ou_code', 'operation_day'], how = 'left').fillna(0)

In [111]:
df_final[df_final['dis_core_os']!=0]

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1,outbound_shipped_qty,kernal_core2,kernal_value2,total_working_hour,kernal_core3,kernal_value3,dis_core,outbound_inbound_qty_ratio,working_hour_per_head,total_head_count,is_holiday,outsource_working_hour,kernal_core4,kernal_value4,dis_core_os
133,CN-002,20210625,0.0,-1,0.000000e+00,12437.0,1,12440.139151,40.42,1,40.870852,0.451,0.000000,8.084000,5.0,0,16.30,0.0,16.369848,0.070
134,CN-002,20210708,0.0,-1,0.000000e+00,5893.0,3,6737.728291,57.73,2,55.509891,-2.220,0.000000,11.546000,5.0,0,21.70,2.0,21.703955,0.004
137,CN-002,20210826,0.0,-1,0.000000e+00,6866.0,3,6737.728291,32.70,4,33.208525,0.509,0.000000,8.175000,4.0,0,16.52,0.0,16.369848,-0.150
141,CN-002,20210903,0.0,-1,0.000000e+00,2955.0,0,2215.048520,40.55,1,40.870852,0.321,0.000000,8.110000,5.0,0,16.35,0.0,16.369848,0.020
142,CN-002,20210707,0.0,-1,0.000000e+00,2514.0,0,2215.048520,57.68,2,55.509891,-2.170,0.000000,11.536000,5.0,0,21.88,2.0,21.703955,-0.176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6919,CN-417,20210829,2881509.0,0,2.881509e+06,0.0,-1,0.000000,206.68,2,208.854467,2.174,0.000000,12.917500,16.0,1,195.50,0.0,186.724177,-8.776
6920,CN-417,20210903,106909.0,4,1.028484e+05,501.0,3,384.390558,88.41,1,56.422029,-31.988,0.004686,8.841000,10.0,0,63.00,1.0,70.209677,7.210
6921,CN-417,20210803,100000.0,4,1.028484e+05,0.0,-1,0.000000,202.25,2,208.854467,6.604,0.000000,9.630952,21.0,0,193.50,0.0,186.724177,-6.776
6922,CN-417,20210824,151671.0,3,1.426260e+05,153.0,0,155.783920,177.15,0,184.355871,7.206,0.001009,8.435714,21.0,0,167.50,4.0,163.938735,-3.561


In [ ]:
# model = sm.OLS(Y = X)
# results = model.fit()
# results.params

In [49]:
import seaborn as sns

import plotly.graph_objects as go
import plotly.express as px

# wide_df = px.data.medals_wide()

# fig = px.bar(wide_df, x="nation", y=["gold", "silver", "bronze"], title="Wide-Form Input")

In [ ]:
plt

In [62]:
from datetime import datetime
import time

plt = df_final[df_final['qt_75_wh']!=0]
plt['operation_day'] = [datetime.strptime(i,'%Y%m%d').strftime('%Y%m%d') for i in plt['operation_day'].astype(str)]
plt['operation_day']= pd.to_datetime(plt['operation_day'])

In [61]:
# # # sns.barplot(x = 'operation_day', y = ['inbound_receive_qty', 'outbound_shipped_qty'],  \
# # #     data=df_final.iloc[0:30,:], color = 'red'
# # #     ) 
# # # # g.set(xlim=(1, 20210805))  
# g = px.bar(plt[plt['ou_code'] == 'CN-298'],\
#     x = 'operation_day', y = ['inbound_receive_qty', 'outbound_shipped_qty']);g


In [63]:
plt2 = plt[plt['ou_code'] == 'CN-305']
plt2['outsource_working_hour'] = plt2['outsource_working_hour'] * 100
plt2 = plt2.sort_values('operation_day')

In [64]:
g = go.Figure([go.Bar(x = plt2['operation_day'] ,y = plt2['inbound_receive_qty']),\
    go.Bar(x = plt2['operation_day'], y = plt2['outbound_shipped_qty'])])
g.update_layout(barmode='stack')
g.add_trace(go.Scatter(x =plt2['operation_day'], y = plt2['outsource_working_hour'], mode ='lines'))
# g.add_trace(go.Scatter(plt[plt['ou_code'] == 'CN-298'], x = 'operation_day', y =  'outsource_working_hour'))

In [65]:
# def mutatation_(new_col, calculate_methdo):
#     df_final[new_col] = df_final.groupby(
#     ['ou_code', 'kernal_core1', 'kernal_core2']
#     )['total_working_hour'].transform(calculate_methdo)
#     return df_final


In [66]:
plt2['dis_core'].describe()

count    151.000000
mean      -0.243781
std       12.303399
min      -37.523000
25%       -9.113000
50%        1.483000
75%        8.670500
max       27.736000
Name: dis_core, dtype: float64

In [67]:
px.box(plt2, y = ['dis_core',  'dis_core_os'])

In [ ]:
go.Figure([go.Histogram(x = plt2['dis_core']),\
    go.Histogram(x= plt2['dis_core_os'])])
 

In [58]:

df_final.head()
df_final['max_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('max')
df_final['min_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('min')
df_final['median_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('median')
df_final['mean_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('mean')
df_final['qt_66_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('quantile', .6667)
df_final['qt_75_wh'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['total_working_hour'].transform('quantile', .75)
df_final['d_to_core_outer'] = df_final.groupby(
        ['ou_code', 'kernal_core1', 'kernal_core2'], as_index = False
                )['total_working_hour','kernal_value3'].diff(axis = 1
                    ).drop('total_working_hour', axis = 1).round(3)
df_final['d_to_core_outer_os'] = df_final.groupby(
        ['ou_code', 'kernal_core1', 'kernal_core2'], as_index = False
                )['outsource_working_hour','kernal_value4'].diff(axis = 1
                    ).drop('outsource_working_hour', axis = 1).round(3)
df_final['d_to_core_outer_os'] = np.abs(df_final['d_to_core_outer_os'])

df_final['percent_error_66'] = (
        df_final['qt_66_wh'] - df_final['total_working_hour'])/(
                df_final['total_working_hour']
                )
df_final['percent_error_75'] = (
        df_final['qt_75_wh'] - df_final['total_working_hour']
        )/(df_final['total_working_hour']
        )

df_final['qt_75_os'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['outsource_working_hour'].transform('quantile', .75)
    

df_final['pe_66_os'] = (
        df_final['qt_66_wh'] - df_final['outsource_working_hour'])/(
                df_final['outsource_working_hour']
                )
df_final['pe_75_os'] = (
        df_final['qt_75_wh'] - df_final['outsource_working_hour']
        )/(df_final['outsource_working_hour']
        )


df_final['qt_75_dis_core_os_inner'] = df_final.groupby(
    ['ou_code', 'kernal_core4']
    )['dis_core_os'].transform('quantile', .75)

df_final['qt_75_dis_core_os_outer'] = df_final.groupby(
    ['ou_code', 'kernal_core1', 'kernal_core2']
    )['d_to_core_outer_os'].transform('quantile', .75)
# 
# df_final['flag_wh'] = [1 if df_final['dis_core_os'][i]>df_final['qt_75_dis_core_os_outer'][i]\
    #  else 0 for i in np.arange(0, len(df_final))]


# df_final['flag_75_wh'] = [1 if df_final['total_working_hour'][i]-1.2*np.abs(df_final['qt_75_wh'][i]) > 0\
#     else 0 for i in np.arange(0, len(df_final))]

# df_final['date_stamp'] = str(date.today())
# df_final['date_stamp'] = df_final['date_stamp'].str.replace('-', '')
 
# df_final['inc_day']  = '99991231'
df_final =df_final.replace(float('inf'), 0); df_final

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1,outbound_shipped_qty,kernal_core2,kernal_value2,total_working_hour,kernal_core3,...,kernal_value4,dis_core_os,d_to_core_outer_os,percent_error_66,percent_error_75,qt_75_os,pe_66_os,pe_75_os,qt_75_dis_core_os_inner,qt_75_dis_core_os_outer
0,CN-001,20210430,0.0,-1,0.000000,1699.0,2,2782.503704,32.75,1,...,0.0,0.0,0.0,0.323080,0.382290,0.0,0.0,0.0,0.0,0.0
1,CN-001,20210619,0.0,-1,0.000000,0.0,-1,0.000000,30.00,1,...,0.0,0.0,0.0,0.202337,0.251167,0.0,0.0,0.0,0.0,0.0
2,CN-001,20210721,0.0,-1,0.000000,1542.0,2,2782.503704,45.17,0,...,0.0,0.0,0.0,-0.040716,0.002214,0.0,0.0,0.0,0.0,0.0
3,CN-001,20210724,0.0,-1,0.000000,0.0,-1,0.000000,9.50,2,...,0.0,0.0,0.0,2.796854,2.951053,0.0,0.0,0.0,0.0,0.0
4,CN-001,20210811,0.0,-1,0.000000,4327.0,2,2782.503704,45.72,0,...,0.0,0.0,0.0,-0.052256,-0.009843,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8144,HK-115,20210611,34955.0,3,39025.502183,5318.0,2,6272.401310,59.10,3,...,0.0,0.0,0.0,0.046026,0.051777,0.0,0.0,0.0,0.0,0.0
8145,HK-115,20210720,58319.0,2,60992.624413,6729.0,2,6272.401310,69.37,1,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
8146,HK-115,20210615,65429.0,2,60992.624413,3516.0,3,2429.029316,75.55,1,...,0.0,0.0,0.0,-0.143026,-0.107280,0.0,0.0,0.0,0.0,0.0
8147,HK-115,20210830,62542.0,2,60992.624413,28971.0,1,27494.223301,84.07,4,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [59]:


df_final['flag_75_wh'] =  df_final['total_working_hour'] -1.2*df_final['qt_75_wh']
df_final['flag_75_wh'] = [1 if a > 0 else 0 for a in df_final['flag_75_wh']]

In [128]:
df_final[df_final['flag_75_wh'] == 1]

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1,outbound_shipped_qty,kernal_core2,kernal_value2,total_working_hour,kernal_core3,...,percent_error_75,qt_75_os,pe_66_os,pe_75_os,qt_75_dis_core_os_inner,qt_75_dis_core_os_outer,flag_75_wh,dis_tt_kernel_x,dis_tt_kernel_y,dis_tt_kernel_y
19,CN-001,20210901,0.0,-1,0.00000,0.0,-1,0.000000,55.07,3,...,-0.318413,0.0,0.0,0.0,0.0,0.0,1,0.0,1.771026,0.0
28,CN-001,20210827,0.0,-1,0.00000,1378.0,2,2782.503704,63.10,3,...,-0.282567,0.0,0.0,0.0,0.0,0.0,1,0.0,-6.258974,0.0
35,CN-001,20210816,0.0,-1,0.00000,0.0,-1,0.000000,56.13,3,...,-0.331285,0.0,0.0,0.0,0.0,0.0,1,0.0,0.711026,0.0
42,CN-001,20210824,0.0,-1,0.00000,0.0,-1,0.000000,46.75,0,...,-0.197112,0.0,0.0,0.0,0.0,0.0,1,0.0,-1.140566,0.0
50,CN-001,20210826,0.0,-1,0.00000,12511.0,0,10264.813592,57.08,3,...,-0.177426,0.0,0.0,0.0,0.0,0.0,1,0.0,-0.238974,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8053,HK-115,20210515,0.0,-1,0.00000,0.0,-1,0.000000,47.14,0,...,-0.257902,0.0,0.0,0.0,0.0,0.0,1,0.0,-5.374635,0.0
8069,HK-115,20210516,0.0,-1,0.00000,0.0,-1,0.000000,65.08,3,...,-0.462469,0.0,0.0,0.0,0.0,0.0,1,0.0,-4.931305,0.0
8078,HK-115,20210713,0.0,-1,0.00000,5437.0,2,6272.401310,94.66,4,...,-0.176130,0.0,0.0,0.0,0.0,0.0,1,0.0,-7.701692,0.0
8091,HK-115,20210813,0.0,-1,0.00000,2794.0,3,2429.029316,78.24,1,...,-0.242683,0.0,0.0,0.0,0.0,0.0,1,0.0,-5.505955,0.0


In [116]:

 
diff_tt_kn = df_final[df_final['flag_75_wh'] == 1][['total_working_hour', 'kernal_value3']].diff(
    axis = 1).drop('total_working_hour', axis = 1)
diff_tt_kn = pd.concat([diff_tt_kn.rename({'kernal_value3' : 'dis_tt_kernel'}, axis = 1), \
    df_final[df_final['flag_75_wh'] == 1]], axis = 1)[['dis_tt_kernel', 'ou_code', 'operation_day']]

df_final = df_final.merge(diff_tt_kn, on = ['ou_code', 'operation_day'], how = 'left').fillna(0)



In [90]:
df_final[df_final['dis_tt_kernel'] != 0]

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1,outbound_shipped_qty,kernal_core2,kernal_value2,total_working_hour,kernal_core3,...,percent_error_66,percent_error_75,qt_75_os,pe_66_os,pe_75_os,qt_75_dis_core_os_inner,qt_75_dis_core_os_outer,flag_wh,flag_75_wh,dis_tt_kernel
389,CN-007,20210613,0.0,-1,0.000000,0.0,-1,0.000000,8.28,1,...,10.898774,14.625000,0.0,0.000000,0.000000,0.000,0.0,0,1,24.567250
396,CN-007,20210411,0.0,-1,0.000000,0.0,-1,0.000000,9.30,1,...,9.593747,12.911290,0.0,0.000000,0.000000,0.000,0.0,0,1,23.547250
400,CN-007,20210711,0.0,-1,0.000000,0.0,-1,0.000000,9.32,1,...,9.571013,12.881438,0.0,0.000000,0.000000,0.000,0.0,0,1,23.527250
1005,CN-035,20210411,0.0,-1,0.000000,0.0,-1,0.000000,11.42,0,...,27.017979,30.910245,0.0,0.000000,0.000000,0.000,0.0,0,1,11.460676
1012,CN-035,20210710,0.0,-1,0.000000,0.0,-1,0.000000,19.30,0,...,15.578514,17.881606,0.0,31.222087,35.698389,3.184,0.0,1,1,3.580676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,CN-428,20210502,6.0,3,211.660751,3.0,0,184.961963,4.50,1,...,8.023025,8.231111,0.0,0.000000,0.000000,0.000,0.0,0,1,3.542022
7468,HK-043,20210814,0.0,-1,0.000000,0.0,-1,0.000000,5.50,1,...,13.394752,13.843182,0.0,0.000000,0.000000,0.000,0.0,0,1,6.337959
7503,HK-043,20210822,0.0,-1,0.000000,0.0,-1,0.000000,2.00,1,...,38.585567,39.818750,0.0,0.000000,0.000000,0.000,0.0,0,1,9.837959
7519,HK-043,20210724,0.0,-1,0.000000,0.0,-1,0.000000,4.50,1,...,16.593585,17.141667,0.0,0.000000,0.000000,0.000,0.0,0,1,7.337959


In [75]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [76]:
df_final.head()

,ou_code,operation_day,inbound_receive_qty,kernal_core1,kernal_value1,outbound_shipped_qty,kernal_core2,kernal_value2,total_working_hour,kernal_core3,...,kernal_value4,dis_core_os,d_to_core_outer_os,percent_error_66,percent_error_75,qt_75_os,pe_66_os,pe_75_os,qt_75_dis_core_os_inner,qt_75_dis_core_os_outer
0,CN-001,20210430,0.0,-1,0.0,1699.0,2,2782.503704,32.75,1,...,0.0,0.0,0.0,0.323080,0.382290,0.0,0.0,0.0,0.0,0.0
1,CN-001,20210619,0.0,-1,0.0,0.0,-1,0.000000,30.00,1,...,0.0,0.0,0.0,0.202337,0.251167,0.0,0.0,0.0,0.0,0.0
2,CN-001,20210721,0.0,-1,0.0,1542.0,2,2782.503704,45.17,0,...,0.0,0.0,0.0,-0.040716,0.002214,0.0,0.0,0.0,0.0,0.0
3,CN-001,20210724,0.0,-1,0.0,0.0,-1,0.000000,9.50,2,...,0.0,0.0,0.0,2.796854,2.951053,0.0,0.0,0.0,0.0,0.0
4,CN-001,20210811,0.0,-1,0.0,4327.0,2,2782.503704,45.72,0,...,0.0,0.0,0.0,-0.052256,-0.009843,0.0,0.0,0.0,0.0,0.0


In [78]:
res = smf.ols('total_working_hour ~ inbound_receive_qty + outbound_shipped_qty', data=df_final).fit()

In [79]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:     total_working_hour   R-squared:                       0.221
Model:                            OLS   Adj. R-squared:                  0.221
Method:                 Least Squares   F-statistic:                     1157.
Date:                Tue, 07 Sep 2021   Prob (F-statistic):               0.00
Time:                        10:57:08   Log-Likelihood:                -53198.
No. Observations:                8149   AIC:                         1.064e+05
Df Residuals:                    8146   BIC:                         1.064e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              145.7991 

In [ ]:

# df_final['percent_error_66'] = (
#         df_final['qt_66_wh'] - df_final['total_working_hour'])/(
#                 df_final['total_working_hour']
#                 )
# df_final['percent_error_75'] = (
#         df_final['qt_75_wh'] - df_final['total_working_hour']
#         )/(df_final['total_working_hour']
#         )

# # df_final = df_final.assign(percent_error_66 = [a-1 if a == 1 else a for a in df_final['percent_error_66']])
# # df_final = df_final.assign(percent_error_75 = [a-1 if a == 1 else a for a in df_final['percent_error_75']])


In [ ]:
# df_final[df_final['percent_error_75'] != 0].dropna()
from datetime import date
df_final['inc_day'] = str(date.today())
df_final['inc_day'] = df_final['inc_day'].str.replace('-', '')

In [ ]:
# df_final.to_csv('./wh_in_out_fin.csv', index= None)

In [ ]:
df_final

---

---

----

In [ ]:
tes = df_final.merge(df_final.groupby('ou_code').agg({
    'inbound_receive_qty': ['std'],
    'outbound_shipped_qty': ['std'],
    'outsource_working_hour': ['std']
    }
).reset_index(), on = 'ou_code', how = 'left')


# .agg({'RECEIPT_ID':['nunique'], 
#         'TOTAL_LINES': ['sum'], 
#         'TOTAL_CONTAINERS': 'sum', 
#         'TOTAL_WEIGHT':'sum',
#         'TOTAL_QTY': ['sum'],
#         'CREATION_DATE_TIME_STAMP_2': 'min' }).reset_index()

In [ ]:
tes = df_final.groupby('ou_code').agg({
    'inbound_receive_qty': ['std'],
    'outbound_shipped_qty': ['std'],
    'outsource_working_hour': ['std']
    }
).reset_index()

In [ ]:
%%time

str(date.today()).replace('-', '')

In [ ]:
df_final

In [ ]:
df_final['kernal_core1'].astype(str) + df_final['kernal_core2'].astype(str) 

In [ ]:
 import plotly.express as px
 def plot3d(ou_code):
    df_plot = df_final[df_final['ou_code'] == ou_code]
    df_plot['kernal_core5'] = df_final['kernal_core1'].astype(str) + df_final['kernal_core2'].astype(str) 
    fig = px.scatter_3d(df_plot, x='inbound_receive_qty', y='outbound_shipped_qty', z='total_working_hour',
              color = 'kernal_core3')
    return fig.show()


In [ ]:
plot3d('CN-214')

In [ ]:

fig.show()